In [107]:
import pandas as pd
import json
# from ease_of_movement import ease_of_movement

In [108]:
file = r'../data/ease_of_movement.csv'
headers = ['date','high','low','volume','emv','sma_14_emv']

df = pd.read_csv(file,
                 names=headers, # Rename the headers
                 header=0)      # Don't use the defined csv headers
df

,date,high,low,volume,emv,sma_14_emv
1,18-Jun-12,63.74,62.63,32178836,NaN,NaN
2,19-Jun-12,64.51,63.85,36461672,1.80,NaN
3,20-Jun-12,64.57,63.81,51372680,0.01,NaN
4,21-Jun-12,64.31,62.62,42476356,-2.88,NaN
5,22-Jun-12,63.43,62.73,29504176,-0.91,NaN
6,25-Jun-12,62.85,61.95,33098600,-1.85,NaN
7,26-Jun-12,62.70,62.06,30577960,-0.04,NaN
8,27-Jun-12,63.18,62.69,35693928,0.76,NaN
9,28-Jun-12,62.47,61.54,49768136,-1.74,NaN
10,29-Jun-12,64.16,63.21,44759968,3.57,NaN


In [109]:
# Convert DataFrame rows into json records
j = df.to_json(orient='records')

In [110]:
# Load as collection
data = json.loads(j)
data[0]

{'date': '18-Jun-12',
 'high': 63.74,
 'low': 62.63,
 'volume': 32178836,
 'emv': None,
 'sma_14_emv': None}

In [111]:
def emv(prev, curr):
    high, low, volume = curr['high'], curr['low'], curr['volume']
    prior_high, prior_low = prev['high'], prev['low']

    # Distance Moved = ((H + L)/2 - (Prior H + Prior L)/2) 
    distance_moved = (high + low) / 2 - (prior_high + prior_low) / 2

    # Box Ratio = ((V/100,000,000)/(H - L))
    box_ratio = (volume / 100000000) / (high - low)
    
    # 1-Period EMV = ((H + L)/2 - (Prior H + Prior L)/2) / ((V/100,000,000)/(H - L))
    return distance_moved / box_ratio

In [112]:
def sma(arr):
    return sum(arr) / len(arr)

In [129]:
# Compute the emv for the given array of stocks, and includes 2 new fields for the computed emv
def compute_emv(data, field1='emv_computed', field2='sma_14_emv_computed'):
    # Computes the emv starting from index 1, since the prior high and low of the stocks is required
    for i, item in enumerate(data):
        # Start at index 1, since prior is required
        if i > 0:
            item[field1] = emv(data[i - 1], item)

        # Start from the 14th index, which is the 15th items
        if i >= 14:
            # Take the last 14 stocks to compute emv, 
            # inclusive of the stocks at current index
            prev_data = data[i-13:i+1]
            emvs = [d.get(field1, 0) 
                    for d in prev_data]
            item[field2] = sma(emvs)
    return data

output = compute_emv(data)

# Convert json back to DataFrame
pd.read_json(json.dumps(output), orient='records')

,date,emv,emv_computed,high,low,sma_14_emv,sma_14_emv_computed,volume
0,2012-06-18,NaN,NaN,63.74,62.63,NaN,NaN,32178836
1,2012-06-19,1.80,1.801069,64.51,63.85,NaN,NaN,36461672
2,2012-06-20,0.01,0.014794,64.57,63.81,NaN,NaN,51372680
3,2012-06-21,-2.88,-2.884546,64.31,62.62,NaN,NaN,42476356
4,2012-06-22,-0.91,-0.913430,63.43,62.73,NaN,NaN,29504176
5,2012-06-25,-1.85,-1.849021,62.85,61.95,NaN,NaN,33098600
6,2012-06-26,-0.04,-0.041860,62.70,62.06,NaN,NaN,30577960
7,2012-06-27,0.76,0.761894,63.18,62.69,NaN,NaN,35693928
8,2012-06-28,-1.74,-1.737859,62.47,61.54,NaN,NaN,49768136
9,2012-06-29,3.57,3.565686,64.16,63.21,NaN,NaN,44759968
